## Geokodiranje adresa i računanje udaljenosti

U ovom dijelu vršimo geokodiranje adresa i računanje udaljenosti od centra grada koristeći geografske koordinate geografske širine i dužine. Izvršavaju se sljedeći koraci:

1. **Geokodiranje adrese**: Definiramo funkciju `geocode_address` koja prima adresu kao ulaz i koristi Nominatim API za dobijanje geografskih koordinata (geografske širine i dužine) adrese. Funkcija šalje zahtjev API-ju, obrađuje JSON odgovor i izvlači vrijednosti geografske širine i dužine. Ako je uspješno, funkcija vraća geografsku širinu i dužinu kao decimalne brojeve. Ako nema rezultata ili se dogodi greška, prikazuju se odgovarajuće poruke o greškama.

2. **Računanje dužine rute**: Definiramo funkciju `get_route_length` koja računa dužinu rute u metrima između dvije skupine geografskih koordinata (geografske širine i dužine) koristeći OSRM API. Funkcija konstruiše URL za API sa startnim i krajnjim koordinatama, šalje zahtjev, obrađuje JSON odgovor i dobija vrijednost dužine rute. Ako je uspješno, funkcija vraća dužinu rute u metrima. Ako nema pronađenih ruta ili se dogodi greška, prikazuju se odgovarajuće poruke o greškama.

3. **Primjer korištenja**: Pružamo primjere kako koristiti funkcije geokodiranja i računanja dužine rute. U prvom primjeru geokodiramo zadatu adresu koristeći funkciju `geocode_address` i ispisujemo geografske koordinate (geografsku širinu i dužinu). U drugom primjeru računamo dužinu rute u metrima između dvije skupine koordinata koristeći funkciju `get_route_length` i ispisujemo rezultat.

4. **Čitanje skupa podataka**: Čitamo CSV datoteku sa podacima (`svi_stanovi.csv`) u DataFrame koristeći funkciju `pd.read_csv`.

5. **Dodavanje kolona za koordinate i udaljenost**: Dodajemo tri nove kolone u DataFrame: `object_latitude` (geografska širina objekta), `object_longitude` (geografska dužina objekta) i `udaljenost_od_centra` (udaljenost od centra grada). Ove kolone će sadržavati geokodirane vrijednosti geografske širine, geografske dužine i izračunate udaljenosti, redom.

6. **Geokodiranje adresa u skupu podataka**: Iteriramo kroz svaki redak u DataFrame-u i geokodiramo adresu koristeći funkciju `geocode_address`. Ako dobijemo važeće vrijednosti geografske širine i dužine, ažuriramo odgovarajuće kolone u DataFrame-u sa geokodiranim vrijednostima.

7. **Računanje dužine vožnje rute**: Iteriramo kroz svaki redak u DataFrame-u i računamo dužinu vožnje rute u metrima od lokacije objekta (geografska širina i dužina) do koordinata centra grada. Ako dobijemo važeću vrijednost dužine rute, ažuriramo kolonu `udaljenost_od_centra` u DataFrame-u. Za centar grada uzeli smo koordinate Baščaršije.

Ovaj postupak omogućava nam mapiranje adresa na geografske koordinate i računanje udaljenosti od centra grada, pružajući vrijedne informacije o lokaciji za daljnju analizu i donošenje odluka.


In [134]:
import requests
import pandas as pd

In [147]:
def geocode_address(address):
    """
    Geocodes the given address using Nominatim API and returns the latitude and longitude.
    """
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": address,
        "format": "json",
        "limit": 1
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data:
            result = data[0]
            latitude = float(result["lat"])
            longitude = float(result["lon"])
            return latitude, longitude
        else:
            print("No results found for the address.")
    else:
        print("Error occurred during geocoding. Status code:", response.status_code)

    return None


def get_route_length(start_lat, start_lon, end_lat, end_lon):
    """
    Calculates the route length in meters using the OSRM API.
    """
    base_url = "https://router.project-osrm.org/route/v1/driving/{},{};{},{}"
    url = base_url.format(start_lon, start_lat, end_lon, end_lat)

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if "routes" in data and len(data["routes"]) > 0:
            route_length = data["routes"][0]["distance"]  # Distance in meters
            return route_length
        else:
            print("No routes found.")
    else:
        print("Error occurred while retrieving the route. Status code:", response.status_code)

    return None

In [148]:
# Example usage
address = "Hakije Kulenovića, Sarajevo"
latitude, longitude = geocode_address(address)
if latitude and longitude:
    print("Latitude:", latitude)
    print("Longitude:", longitude)

Latitude: 43.8636383
Longitude: 18.4077222


In [149]:
# Example usage
start_latitude = 43.849374
start_longitude = 18.397530
end_latitude = 43.8599809478476
end_longitude = 18.431355841003494

route_length = get_route_length(start_latitude, start_longitude, end_latitude, end_longitude)
if route_length is not None:
    print("Route Length (meters):", route_length)

Route Length (meters): 3813.9


In [153]:
df = pd.read_csv('svi_stanovi.csv', sep=",")

In [154]:
df['object_latitude'] = 0
df['object_longitude'] = 0
df['udaljenost_od_centra'] = 0

In [155]:
# Geocode address column:

for index, row in df.iterrows():
    address = row['adresa'] 
    if pd.isna(address):
        pass
    else:
        print(index)
        response = geocode_address(address+ ', Sarajevo')
        if type(response)==tuple:
            
            df.loc[index, 'object_latitude'] = response[0]
            df.loc[index, 'object_longitude'] = response[1]

0
1
3
4
14
16
17
19
21
22
23
25
26
27
29
33
34
35
No results found for the address.
37
38
40
41
43
45
46
47
48
49
51
52
53
54
55
56
57
58
59
63
64
66
67
68
69
71
73
74
76
77
78
81
82
83
84
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
106
107
108
109
110
111
112
113
114
115
116
118
119
120
122
124
125
127
133
135
136
137
138
139
141
142
144
146
147
148
149
150
151
152
154
156
159
160
161
162
164
165
168
170
172
173
174
175
177
180
No results found for the address.
181
182
183
184
185
186
187
188
189
191
192
193
194
195
196
197
199
200
201
202
204
206
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
232
234
235
237
241
243
244
245
246
247
249
250
251
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
278
279
280
281
282
285
286
287
288
289
290
291
292
293
294
295
297
298
299
301
302
303
304
305
307
308
310
311
312
314
315
317
318
319
320
322
323
324
327
329
331
332
333
338
340
347
348
351
352
355
360

In [156]:
# Calculate driving route length in meters from object to Bascarsija
# Bascarsija coordinates:
end_latitude = 43.8599809478476
end_longitude = 18.431355841003494

for index, row in df.iterrows():
    start_latitude = row['object_latitude'] 
    start_longitude = row['object_longitude'] 
    if start_latitude!=0:
        print(index)
        route_length = get_route_length(start_latitude, start_longitude, end_latitude, end_longitude)
        if route_length is not None:
            # print("Route Length (meters):", route_length)
            df.loc[index, 'udaljenost_od_centra'] = route_length

0
1
3
4
14
16
17
19
21
22
23
25
26
27
29
33
34
37
38
40
41
43
45
46
47
48
49
51
52
53
54
55
56
57
58
59
63
64
66
67
68
69
71
73
74
76
77
78
81
82
83
84
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
106
107
108
109
110
111
112
113
114
115
116
118
119
120
122
124
125
127
133
135
136
137
138
139
141
142
144
146
147
148
149
150
151
152
154
156
159
160
161
162
164
165
168
170
172
173
174
175
177
181
182
183
184
185
186
187
188
189
191
192
193
194
195
196
197
199
200
201
202
204
206
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
232
234
235
237
241
243
244
245
246
247
249
250
251
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
278
279
280
281
282
285
286
287
288
289
290
291
292
293
294
295
297
298
299
301
302
303
304
305
307
308
310
311
312
314
315
317
318
319
320
322
323
324
327
329
331
332
333
338
340
347
348
351
352
355
360
363
365
366
367
369
371
372
373
374
377
378
379
383
384
385
387
388
390
39

In [157]:
df.to_csv('part_geocoded.csv')